In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import plotly.graph_objects as go


## SALUD

In [2]:
dir_input = Path("data/base100/")
ipc_file = dir_input / "base_salud.xlsx"

df_salud = pd.read_excel(ipc_file)

# Asegurarse de que 'VALOR' esté en formato numérico
df_salud["VALOR"] = pd.to_numeric(df_salud["VALOR"], errors="coerce")

# Asumir que las categorías son las mismas
categorias = df_salud["CATEGORIA"].unique().tolist()

# Pesos  proporcionados
pesos_manual = [
    0.016, 0.159, 0.137, 0.107, 0.065, 0.182, 0.058, 0.085, 0.024, 0.237
]

# NORMALIZACIÓN 
w_salud = 3.475

suma_original = sum(pesos_manual)
factor = w_salud / suma_original
pesos_norm = [p * factor for p in pesos_manual]

# print("\nSuma original:", suma_original)
# print("Suma normalizada:", sum(pesos_norm))

# Crear dataframe de pesos
df_pesos = pd.DataFrame({
    "CATEGORIA": categorias,
    "PESO": pesos_manual,
    "PESO_NORMALIZADO": pesos_norm
})


# Convertir los pesos normalizados a una matriz NumPy
w = np.array(pesos_norm)

# Función de ponderación geométrica
def geom_pond(x, w):
    x = np.array(x)
    return np.exp(np.sum(w * np.log(x)) / np.sum(w))

# Crear el diccionario de IPC por fecha
ipc_salud = {}

# Asegurarse de que 'FECHA' sea una columna válida en df_salud, y ordenarla por fecha
for fecha in df_salud["FECHA"].unique(): 
    # Filtramos los datos para esa fecha y agrupamos por 'CATEGORIA'
    valores = df_salud[df_salud["FECHA"] == fecha]["VALOR"].values
    ipc_salud[fecha] = geom_pond(valores, w)  # Usamos los valores de la columna 'VALOR'

# Convertir a una serie final ordenada por fecha
ipc_salud = pd.Series(ipc_salud)
ipc_salud.index = pd.to_datetime(ipc_salud.index)
ipc_salud = ipc_salud.sort_index()

#######################################################################
# Convertir ipc_salud a DataFrame para concatenarlo con df_salud
df_ipc_salud = ipc_salud.reset_index()
df_ipc_salud.columns = ["FECHA", "VALOR"]

# Agregar un identificador de categoría general para el IPC
df_ipc_salud["CATEGORIA"] = "Índice Salud"

# Concatenar los datos originales con la fila calculada para IPC
df_salud = pd.concat([df_salud, df_ipc_salud], ignore_index=True)

df_salud

import plotly.graph_objects as go

# Crear el gráfico
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=ipc_salud.index,
    y=ipc_salud.values,
    mode="lines",
    name="Índice (Base 100)",
    line=dict(width=2)
))

fig.update_layout(
    title=None,
    xaxis_title="Fecha",
    yaxis_title="Índice (Base 100)",
    template="plotly_white",
    hovermode="x unified",
    width=900,
    height=500
)

fig.show()
fig.write_image("results/indice_salud.pdf")

## ALIMENTOS 

In [3]:
dir_input = Path("data/base100/")
ipc_file = dir_input / "base_alimentos.xlsx"

df_alimentos = pd.read_excel(ipc_file)

df_alimentos["VALOR"] = pd.to_numeric(df_alimentos["VALOR"], errors="coerce")

categorias_alimentos = df_alimentos["CATEGORIA"].unique().tolist()

# Pesos manuales para la división de alimentos
pesos_alimentos = [
    0.384, 1.373, 0.268, 0.103, 1.210, 0.369, 0.206, 3.070, 1.346,
    0.182, 0.184, 0.192, 0.354, 0.354, 2.425, 0.151, 0.096, 0.255,
    0.619, 0.706, 1.303, 0.022, 0.320, 0.075, 0.322, 0.106, 0.137,
    0.059, 2.134, 0.523, 0.220, 0.211, 0.060, 0.095, 0.316
]

# NORMALIZACIÓN 
w_alimentos = 23.928  # Definir w_alimentos 

suma_original = sum(pesos_alimentos)
factor = w_alimentos / suma_original
pesos_norm_alimentos = [p * factor for p in pesos_alimentos]

# Crear dataframe de pesos para alimentos
df_pesos_alimentos = pd.DataFrame({
    "CATEGORIA": categorias_alimentos,
    "PESO": pesos_alimentos,
    "PESO_NORMALIZADO": pesos_norm_alimentos
})

# Convertir los pesos normalizados a una matriz NumPy
w_alimentos = np.array(pesos_norm_alimentos)

# Función de ponderación geométrica
def geom_pond(x, w):
    x = np.array(x)
    return np.exp(np.sum(w * np.log(x)) / np.sum(w))

# Crear el diccionario de IPC por fecha
ipc_alimentos = {}

# Asegurarse de que 'FECHA' sea una columna válida en df_alimentos
for fecha in df_alimentos["FECHA"].unique(): 
    # Filtramos los datos para esa fecha y agrupamos por 'CATEGORIA'
    valores = df_alimentos[df_alimentos["FECHA"] == fecha]["VALOR"].values
    ipc_alimentos[fecha] = geom_pond(valores, w_alimentos)  # Usamos los valores de la columna 'VALOR'

# Convertir a una serie final ordenada por fecha
ipc_alimentos = pd.Series(ipc_alimentos)
ipc_alimentos.index = pd.to_datetime(ipc_alimentos.index)
ipc_alimentos = ipc_alimentos.sort_index()

# Convertir ipc_alimentos a DataFrame para concatenarlo con df_alimentos
df_ipc_alimentos = ipc_alimentos.reset_index()
df_ipc_alimentos.columns = ["FECHA", "VALOR"]

# Agregar un identificador de categoría general para el IPC de alimentos
df_ipc_alimentos["CATEGORIA"] = "Índice Alimentos"

# Concatenar los datos originales con la fila calculada para IPC
df_alimentos = pd.concat([df_alimentos, df_ipc_alimentos], ignore_index=True)


#######################################################################
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=ipc_alimentos.index,
    y=ipc_alimentos.values,
    mode="lines",
    name="Índice (Base 100)",
    line=dict(width=2)
))

fig.update_layout(
    title=None,
    xaxis_title="Fecha",
    yaxis_title="Índice (Base 100)",
    template="plotly_white",
    hovermode="x unified",
    width=900,
    height=500
)

fig.show()

fig.write_image("results/indice_alimentos.pdf")

## EDUCACIÓN

In [4]:
dir_input = Path("data/base100/")
df_educacion = dir_input / "base_educacion.xlsx"

df_educacion = pd.read_excel(df_educacion)

# Asegurarse de que 'VALOR' esté en formato numérico
df_educacion["VALOR"] = pd.to_numeric(df_educacion["VALOR"], errors="coerce")

# Pesos para la división Educación
pesos_educacion = [
    0.691,
    1.553,
    1.259
]

# Normalización
w_educacion = 8.058  # Peso total de la división

suma_original = sum(pesos_educacion)
factor = w_educacion / suma_original
pesos_norm_educacion = [p * factor for p in pesos_educacion]

# Crear dataframe de pesos
categorias_educacion = df_educacion["CATEGORIA"].unique().tolist()
df_pesos_educacion = pd.DataFrame({
    "CATEGORIA": categorias_educacion,
    "PESO": pesos_educacion,
    "PESO_NORMALIZADO": pesos_norm_educacion
})

# Convertir los pesos normalizados a una matriz NumPy
w_educacion = np.array(pesos_norm_educacion)

# Función de ponderación geométrica
def geom_pond(x, w):
    x = np.array(x)
    return np.exp(np.sum(w * np.log(x)) / np.sum(w))

# Crear el diccionario de IPC por fecha
ipc_educacion = {}
for fecha in df_educacion["FECHA"].unique(): 
    valores = df_educacion[df_educacion["FECHA"] == fecha]["VALOR"].values
    ipc_educacion[fecha] = geom_pond(valores, w_educacion)

# Convertir a una serie final ordenada por fecha
ipc_educacion = pd.Series(ipc_educacion)
ipc_educacion.index = pd.to_datetime(ipc_educacion.index)
ipc_educacion = ipc_educacion.sort_index()

# Concatenar IPC con el dataframe original
df_ipc_educacion = ipc_educacion.reset_index()
df_ipc_educacion.columns = ["FECHA", "VALOR"]
df_ipc_educacion["CATEGORIA"] = "Índice Educación"

# Concatenar datos originales con el IPC
df_educacion = pd.concat([df_educacion, df_ipc_educacion], ignore_index=True)

# Crear el gráfico de IPC de Educación
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=ipc_educacion.index,
    y=ipc_educacion.values,
    mode="lines",
    name="Índice (Base 100)",
    line=dict(width=2)
))

fig.update_layout(
    title=None,
    xaxis_title="Fecha",
    yaxis_title="Índice (Base 100)",
    template="plotly_white",
    hovermode="x unified",
    width=900,
    height=500
)

fig.show()
fig.write_image("results/indice_educacion.pdf")


## TRANSPORTE

In [5]:
dir_input = Path("data/base100/")
df_combustibles = dir_input / "base_combustibles.xlsx"
df_combustibles = pd.read_excel(df_combustibles)

# Asegurarse de que 'VALOR' esté en formato numérico
df_combustibles["VALOR"] = pd.to_numeric(df_combustibles["VALOR"], errors="coerce")

# Pesos para la división Combustibles
pesos_combustibles = [
    12.210
]

# Normalización
w_combustibles = 12.210  # Peso total de la división

suma_original = sum(pesos_combustibles)
factor = w_combustibles / suma_original
pesos_norm_combustibles = [p * factor for p in pesos_combustibles]

# Crear dataframe de pesos
categorias_combustibles = df_combustibles["CATEGORIA"].unique().tolist()
df_pesos_combustibles = pd.DataFrame({
    "CATEGORIA": categorias_combustibles,
    "PESO": pesos_combustibles,
    "PESO_NORMALIZADO": pesos_norm_combustibles
})

# Convertir los pesos normalizados a una matriz NumPy
w_combustibles = np.array(pesos_norm_combustibles)

# Función de ponderación geométrica
def geom_pond(x, w):
    x = np.array(x)
    return np.exp(np.sum(w * np.log(x)) / np.sum(w))

# Crear el diccionario de IPC por fecha
ipc_combustibles = {}
for fecha in df_combustibles["FECHA"].unique(): 
    valores = df_combustibles[df_combustibles["FECHA"] == fecha]["VALOR"].values
    ipc_combustibles[fecha] = geom_pond(valores, w_combustibles)

# Convertir a una serie final ordenada por fecha
ipc_combustibles = pd.Series(ipc_combustibles)
ipc_combustibles.index = pd.to_datetime(ipc_combustibles.index)
ipc_combustibles = ipc_combustibles.sort_index()

# Concatenar IPC con el dataframe original
df_ipc_combustibles = ipc_combustibles.reset_index()
df_ipc_combustibles.columns = ["FECHA", "VALOR"]
df_ipc_combustibles["CATEGORIA"] = "Índice Combustibles"

# Concatenar datos originales con el IPC
df_combustibles = pd.concat([df_combustibles, df_ipc_combustibles], ignore_index=True)

# Crear el gráfico de IPC de Combustibles
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=ipc_combustibles.index,
    y=ipc_combustibles.values,
    mode="lines",
    name="Índice (Base 100)",
    line=dict(width=2)
))

fig.update_layout(
    title=None,
    xaxis_title="Fecha",
    yaxis_title="Índice (Base 100)",
    template="plotly_white",
    hovermode="x unified",
    width=900,
    height=500
)

fig.show()
fig.write_image("results/indice_combustibles.pdf")


## VIVIENDA


In [6]:
dir_input = Path("data/base100/")
ipc_file = dir_input / "base_vivienda.xlsx"
df_vivienda = pd.read_excel(ipc_file)

# Asegurarse de que 'VALOR' esté en formato numérico
df_vivienda["VALOR"] = pd.to_numeric(df_vivienda["VALOR"], errors="coerce")

# Pesos para la división Vivienda
pesos_vivienda = [
    4.451,
    0.000
]

# Normalización
w_vivienda = 9.607  # Peso total de la división

suma_original = sum(pesos_vivienda)
factor = w_vivienda / suma_original
pesos_norm_vivienda = [p * factor for p in pesos_vivienda]

# Crear dataframe de pesos
categorias_vivienda = df_vivienda["CATEGORIA"].unique().tolist()
df_pesos_vivienda = pd.DataFrame({
    "CATEGORIA": categorias_vivienda,
    "PESO": pesos_vivienda,
    "PESO_NORMALIZADO": pesos_norm_vivienda
})

# Convertir los pesos normalizados a una matriz NumPy
w_vivienda = np.array(pesos_norm_vivienda)

# Función de ponderación geométrica
def geom_pond(x, w):
    x = np.array(x)
    return np.exp(np.sum(w * np.log(x)) / np.sum(w))

# Crear el diccionario de IPC por fecha
ipc_vivienda = {}
for fecha in df_vivienda["FECHA"].unique(): 
    valores = df_vivienda[df_vivienda["FECHA"] == fecha]["VALOR"].values
    ipc_vivienda[fecha] = geom_pond(valores, w_vivienda)

# Convertir a una serie final ordenada por fecha
ipc_vivienda = pd.Series(ipc_vivienda)
ipc_vivienda.index = pd.to_datetime(ipc_vivienda.index)
ipc_vivienda = ipc_vivienda.sort_index()

# Concatenar IPC con el dataframe original
df_ipc_vivienda = ipc_vivienda.reset_index()
df_ipc_vivienda.columns = ["FECHA", "VALOR"]
df_ipc_vivienda["CATEGORIA"] = "Índice Vivienda"

# Concatenar datos originales con el IPC
df_vivienda = pd.concat([df_vivienda, df_ipc_vivienda], ignore_index=True)

# Crear el gráfico de IPC de Vivienda
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=ipc_vivienda.index,
    y=ipc_vivienda.values,
    mode="lines",
    name="Índice (Base 100)",
    line=dict(width=2)
))

fig.update_layout(
    title=None,
    xaxis_title="Fecha",
    yaxis_title="Índice (Base 100)",
    template="plotly_white",
    hovermode="x unified",
    width=900,
    height=500
)

fig.show()
fig.write_image("results/indice_vivienda.pdf")


## ÍNDICE FINAL

In [7]:
# Pesos oficiales de cada división
pesos_div = {
    "ALIMENTOS": 23.928,
    "SALUD": 3.475,
    "EDUCACION": 8.058,
    "COMBUSTIBLES": 12.210,
    "VIVIENDA": 9.607
}

# Suma total oficial
suma_total = sum(pesos_div.values())

# Normalización para que sumen 100
factor = 100 / suma_total
pesos_div_norm = {k: v * factor for k, v in pesos_div.items()}

# Convertir a vector en el mismo orden
divisiones = ["ALIMENTOS", "SALUD", "EDUCACION", "COMBUSTIBLES", "VIVIENDA"]
w = np.array([pesos_div_norm[d] for d in divisiones])
w = w / w.sum()  # Por si acaso

# Función de ponderación geométrica
def geom_pond(x, w):
    x = np.array(x)
    return np.exp(np.sum(w * np.log(x)) / np.sum(w))

# Extraer los índices sectoriales de cada dataframe por fecha
# Para cada sector, extraemos el índice sectorial por fecha
ipc_alimentos = df_alimentos[df_alimentos["CATEGORIA"] == "Índice Alimentos"].set_index("FECHA")["VALOR"]
ipc_salud = df_salud[df_salud["CATEGORIA"] == "Índice Salud"].set_index("FECHA")["VALOR"]
ipc_educacion = df_educacion[df_educacion["CATEGORIA"] == "Índice Educación"].set_index("FECHA")["VALOR"]
ipc_combustibles = df_combustibles[df_combustibles["CATEGORIA"] == "Índice Combustibles"].set_index("FECHA")["VALOR"]
ipc_vivienda = df_vivienda[df_vivienda["CATEGORIA"] == "Índice Vivienda"].set_index("FECHA")["VALOR"]

# Crear el dataframe de IPC por divisiones (por fecha)
ipc_divisiones = pd.DataFrame({
    "ALIMENTOS": ipc_alimentos,
    "SALUD": ipc_salud,
    "EDUCACION": ipc_educacion,
    "COMBUSTIBLES": ipc_combustibles,
    "VIVIENDA": ipc_vivienda
})

# Asegurarnos de que todas las fechas coincidan en cada columna
ipc_divisiones = ipc_divisiones.dropna()  # Eliminar fechas con NaN si alguna división no tiene datos en esa fecha

# Calcular IPC total ponderado por fecha
ipc_total = {}

for fecha in ipc_divisiones.index:
    valores = ipc_divisiones.loc[fecha].values
    ipc_total[fecha] = geom_pond(valores, w)

ipc_total = pd.Series(ipc_total)
ipc_total = ipc_total.sort_index()

# Concatenar IPC total con los dataframes de cada sector
df_ipc_total = ipc_total.reset_index()
df_ipc_total.columns = ["FECHA", "VALOR"]
df_ipc_total["CATEGORIA"] = "IPC"

# Concatenar todos los dataframes (sectoriales + IPC total)
df_final = pd.concat([df_salud, df_alimentos, df_educacion, df_combustibles, df_vivienda, df_ipc_total], ignore_index=True)

# Crear el gráfico para el IPC total
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=ipc_total.index,
    y=ipc_total.values,
    mode="lines",
    name="Índice Total (Base 100)",
    line=dict(width=2)
))

fig.update_layout(
    title=None,
    xaxis_title="Fecha",
    yaxis_title="Índice (Base 100)",
    template="plotly_white",
    hovermode="x unified",
    width=900,
    height=500
)

fig.show()

# Guardar el gráfico como un archivo PDF
fig.write_image("results/indice_final.pdf")



c:\Users\jhers\anaconda3\Lib\site-packages\pandas\core\indexes\base.py:7654: FutureWarning:

Dtype inference on a pandas object (Series, Index, ExtensionArray) is deprecated. The Index constructor will keep the original dtype in the future. Call `infer_objects` on the result to get the old behavior.



In [8]:
df_final["CATEGORIA"].unique()

array(['ANTICONCEPTIVOS', 'ANTIINFECCIOSOS SISTÉMICOS',
       'APARATO CARDIOVASCULAR', 'APARATO DIGESTIVO',
       'APARATO GENITOURINARIO Y REPRODUCTOR', 'APARATO RESPIRATORIO',
       'OFTALMOLÓGICOS', 'PIEL Y MUCOSAS',
       'SANGRE, LÍQUIDOS Y ELECTROLITOS', 'SISTEMA NERVIOSO',
       'Índice Salud', 'ACEITES COMESTIBLES',
       'AGUAS MINERALES Y BEBIDAS GASEOSAS',
       'AJÍES, ADEREZOS, SALSAS Y OTROS',
       'ALIMENTOS PROCESADOS E INSUFLADOS (POPEADOS)',
       'ARROZ DE TODAS CLASES', 'AZÚCAR Y OTRAS PRESENTACIONES',
       'BOCADITOS Y ALIMENTOS DIVERSOS',
       'CARNE DE AVE FRESCA, REFRIGERADA Y CONGELADA',
       'CARNE DE RES FRESCA, REFRIGERADA Y CONGELADA',
       'CEREALES EN GRANO', 'CEREALES EN HOJUELA',
       'CHOCOLATE Y PRODUCTOS DE CONFITERÍA', 'CONSERVAS',
       'EMBUTIDO Y CARNE SECA', 'FRUTAS FRESCAS', 'GRASAS COMESTIBLES',
       'HARINA FINA Y HARINA GRUESA',
       'HIELO COMESTIBLE, HELADOS Y SORBETES',
       'HORTALIZAS DE RAÍZ OR BULBO Y HONGO